In [1]:
from itertools import product
import pickle

import numpy as np

from algorithms.nn import NeuralNetwork
from algorithms.preprocessing import load_and_preprocess_dataset

import config

In [2]:
nn_logs_dir = config.LOGS_DIR / 'nn'
nn_logs_dir.mkdir(exist_ok=True)

In [3]:
X, y, X_test, y_test = load_and_preprocess_dataset()

In [4]:
X = np.asarray(X)
y = np.asarray(y)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [5]:
layers_list = [
    [32, 1],
    [64, 1],
    [32, 32, 1],
    [64, 64, 1],
]

In [6]:
alphas = [1e-2, 5e-3, 1e-3, 5e-4]

In [7]:
momentums = [0.85, 0.95]

In [8]:
batch_sizes = [32, 64]

In [9]:
indices = np.random.permutation(len(y))
fold_size = len(indices) // 5
scores = []
for layers, alpha, momentum, batch_size in product(layers_list, alphas, momentums, batch_sizes):
    model = NeuralNetwork(layers)
    fold_scores = []
    for fold in range(5):
        valid_indices = indices[fold * fold_size: (fold + 1) * fold_size]
        train_indices = indices[~np.isin(indices, valid_indices)]

        X_train, y_train = X[train_indices], y[train_indices]
        X_valid, y_valid = X[valid_indices], y[valid_indices]

        history = model.fit(X_train, y_train, X_valid, y_valid, alpha=alpha,
                            batch_size=batch_size, epochs=500, cold_start=True,
                            patience=50, min_delta=1e-3)
        fold_scores.append(history)
    with open(nn_logs_dir / f'alpha_{alpha}-momentum_{momentum}-batch_size_{batch_size}-layers_{"_".join([str(x) for x in layers])}.pkl', 'wb') as f:
        pickle.dump(fold_scores, f)

n_neurons=32-1, alpha=0.01, momentum=0.85, batch_size=32:  24%|██▎       | 118/500 [00:14<00:47,  8.04it/s, train_mse=0.1630026, valid_mse=0.0979939]
n_neurons=32-1, alpha=0.01, momentum=0.85, batch_size=32:  23%|██▎       | 114/500 [00:15<00:51,  7.52it/s, train_mse=0.1283875, valid_mse=0.1061454]
n_neurons=32-1, alpha=0.01, momentum=0.85, batch_size=32:  27%|██▋       | 135/500 [00:19<00:51,  7.07it/s, train_mse=0.1417386, valid_mse=0.1524306]
n_neurons=32-1, alpha=0.01, momentum=0.85, batch_size=32:  26%|██▌       | 129/500 [00:21<01:00,  6.10it/s, train_mse=0.1398672, valid_mse=0.1661201]
n_neurons=32-1, alpha=0.01, momentum=0.85, batch_size=32:  23%|██▎       | 116/500 [00:20<01:08,  5.58it/s, train_mse=0.1227535, valid_mse=0.1700111]
n_neurons=32-1, alpha=0.01, momentum=0.85, batch_size=64:  29%|██▉       | 144/500 [00:13<00:32, 10.99it/s, train_mse=0.2350962, valid_mse=0.1702262]
n_neurons=32-1, alpha=0.01, momentum=0.85, batch_size=64:  26%|██▌       | 131/500 [00:11<00:33, 11.